In [24]:
! pip install xlsxwriter

   ---------------------------------------- 0.0/172.3 kB ? eta -:--:--
   ---------------------------------------- 0.0/172.3 kB ? eta -:--:--
   ---------------------------------------- 0.0/172.3 kB ? eta -:--:--
   -- ------------------------------------- 10.2/172.3 kB ? eta -:--:--
   ---- ---------------------------------- 20.5/172.3 kB 165.2 kB/s eta 0:00:01
   ---- ---------------------------------- 20.5/172.3 kB 165.2 kB/s eta 0:00:01
   --------- ----------------------------- 41.0/172.3 kB 196.9 kB/s eta 0:00:01
   ------------- ------------------------- 61.4/172.3 kB 252.2 kB/s eta 0:00:01
   --------------------------- ---------- 122.9/172.3 kB 450.6 kB/s eta 0:00:01
   -------------------------------------- 172.3/172.3 kB 576.9 kB/s eta 0:00:00



[notice] A new release of pip is available: 24.0 -> 25.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [5]:
import pdfplumber
import re
import pandas as pd
from collections import defaultdict
import glob
import os
import statistics

# ===================== PARÁMETROS AJUSTABLES =====================
FOLDER_PATH = "data/"
OUTPUT_FOLDER = "outputs/"
RIGHT_CUT_FRAC = 0.74          # corte para ignorar columna derecha (x1 <= width * RIGHT_CUT_FRAC)
LEVEL_LEFT_MARGIN_FRAC = 0.08  # para detectar los dígitos grandes de nivel a la izquierda
HEIGHT_MIN_FRAC = 0.88         # descarta "words" cuyo alto < (mediana_alto * este_factor)
MIN_HEIGHT_PX = 7.5            # umbral de seguridad: si la mediana falla, usa este mínimo absoluto
# ================================================================

os.makedirs(OUTPUT_FOLDER, exist_ok=True)

# PDFs a procesar
pdf_files = glob.glob(os.path.join(FOLDER_PATH, "*.pdf"))

# Regex de códigos: 4 letras + 3 dígitos
CODE_RE = re.compile(r"\b[A-Z]{4}\d{3}\b")

# Mapa de símbolos a espacio para normalizar (flechas, guiones, bullets)
ARROW_CHARS = [
    "→","⇒","➔","⟶","⟼","⟿","↦","↠","➤","▶","❯","›","»",
    "-", "—", "–", "·", "•", "‣", "∙", "·"
]
ARROW_TABLE = str.maketrans({ch: " " for ch in ARROW_CHARS})

def normalize_token(txt: str) -> str:
    # Limpia flechas y símbolos, compacta espacios
    txt = (txt or "").translate(ARROW_TABLE)
    txt = re.sub(r"\s+", " ", txt.strip())
    return txt

all_rows = []

for pdf_path in pdf_files:
    file_name = os.path.splitext(os.path.basename(pdf_path))[0]
    levels_codes = defaultdict(list)
    carrera_name = "DESCONOCIDA"

    with pdfplumber.open(pdf_path) as pdf:
        # ------ Carrera ------
        first_text = pdf.pages[0].extract_text() or ""
        m = re.search(r"Carrera:\s*(.+)", first_text, flags=re.IGNORECASE)
        if m:
            carrera_name = m.group(1).strip()

        for page in pdf.pages:
            width, height = page.width, page.height

            # Extraemos palabras con coordenadas
            words = page.extract_words(
                x_tolerance=2, y_tolerance=2,
                keep_blank_chars=False, use_text_flow=True
            )

            # Ignorar columna derecha
            right_cut = width * RIGHT_CUT_FRAC
            left_words = [w for w in words if w["x1"] <= right_cut]

            # ------------------ FILTRO POR TAMAÑO ------------------
            # Calculamos la mediana del alto de los words (proxy de font-size)
            heights = [(w["bottom"] - w["top"]) for w in left_words if w["bottom"] > w["top"]]
            if heights:
                med_h = statistics.median(heights)
                height_threshold = max(med_h * HEIGHT_MIN_FRAC, MIN_HEIGHT_PX)
            else:
                height_threshold = MIN_HEIGHT_PX

            # Dejamos solo words "grandes" (evita códigos chiquitos de flechas/prerrequisitos)
            left_words_big = [w for w in left_words if (w["bottom"] - w["top"]) >= height_threshold]

            # ------------------ DETECCIÓN DE NIVELES ------------------
            level_markers = [
                {"text": w["text"], "y": (w["top"] + w["bottom"]) / 2}
                for w in left_words_big
                if w["text"] in list("123456789") and w["x0"] < width * LEVEL_LEFT_MARGIN_FRAC
            ]
            level_markers.sort(key=lambda d: d["y"])

            # Deduplicar por proximidad vertical
            dedup_levels = []
            for lm in level_markers:
                if not dedup_levels or abs(lm["y"] - dedup_levels[-1]["y"]) > 8:
                    dedup_levels.append(lm)

            # Rango Y por nivel
            ranges = []
            for i, lm in enumerate(dedup_levels):
                y_top = lm["y"] - 24
                y_bottom = (dedup_levels[i+1]["y"] - 24) if (i+1 < len(dedup_levels)) else height
                try:
                    lvl = int(lm["text"])
                    ranges.append((lvl, y_top, y_bottom))
                except:
                    pass

            # ------------------ EXTRACCIÓN DE CÓDIGOS ------------------
            # Normalizamos texto (para no perder códigos pegados a flechas/símbolos)
            for w in left_words_big:
                cleaned = normalize_token(w["text"]).upper()
                # Buscar TODAS las coincidencias (si vienen pegadas)
                for tok in re.findall(r"[A-Z]{4}\d{3}", cleaned):
                    if CODE_RE.fullmatch(tok):
                        y_center = (w["top"] + w["bottom"]) / 2
                        # Asignar por nivel según rango vertical
                        for lvl, y0, y1 in ranges:
                            if y0 <= y_center < y1:
                                levels_codes[lvl].append(tok)
                                break

    # Limpiar duplicados por nivel manteniendo orden
    for lvl in levels_codes:
        seen = set()
        ordered = []
        for c in levels_codes[lvl]:
            if c not in seen:
                ordered.append(c)
                seen.add(c)
        levels_codes[lvl] = ordered

    # Armar filas
    rows = []
    for lvl in sorted(levels_codes.keys()):
        for code in levels_codes[lvl]:
            rows.append({
                "archivo": file_name,
                "carrera": carrera_name,
                "nivel": lvl,
                "codigo": code
            })

    # Guardar por archivo
    df = pd.DataFrame(rows).sort_values(["nivel", "codigo"]).reset_index(drop=True)
    out_csv = os.path.join(OUTPUT_FOLDER, f"{file_name}_codigos_por_nivel.csv")
    df.to_csv(out_csv, index=False)
    print(f"Procesado: {pdf_path} -> {out_csv}")

    all_rows.extend(rows)

# Consolidado
df_all = pd.DataFrame(all_rows).sort_values(["archivo", "nivel", "codigo"]).reset_index(drop=True)
df_all.to_csv(os.path.join(OUTPUT_FOLDER, "todas_mallas_codigos.csv"), index=False)
print("✅ Listo. Consolidado en outputs/todas_mallas_codigos.csv")


Procesado: data\malla_administracion_empresas.pdf -> outputs/malla_administracion_empresas_codigos_por_nivel.csv
Procesado: data\malla_agroindustria.pdf -> outputs/malla_agroindustria_codigos_por_nivel.csv
Procesado: data\malla_ambiental.pdf -> outputs/malla_ambiental_codigos_por_nivel.csv
Procesado: data\malla_ciencia_datos_IA.pdf -> outputs/malla_ciencia_datos_IA_codigos_por_nivel.csv
Procesado: data\malla_civil.pdf -> outputs/malla_civil_codigos_por_nivel.csv
Procesado: data\malla_computacion.pdf -> outputs/malla_computacion_codigos_por_nivel.csv
Procesado: data\malla_economia.pdf -> outputs/malla_economia_codigos_por_nivel.csv
Procesado: data\malla_electricidad.pdf -> outputs/malla_electricidad_codigos_por_nivel.csv
Procesado: data\malla_electronica_automatizacion.pdf -> outputs/malla_electronica_automatizacion_codigos_por_nivel.csv
Procesado: data\malla_fisica.pdf -> outputs/malla_fisica_codigos_por_nivel.csv
Procesado: data\malla_geologia.pdf -> outputs/malla_geologia_codigos_por

In [6]:
import pandas as pd

# Cargar la base de profesores (con varias columnas)
profesores_df = pd.read_excel("data/asignaturas.xlsx")

# Renombramos la columna de código para facilidad
profesores_df = profesores_df.rename(columns={"SII - Código Materia": "codigo"})

# Cargar el consolidado de mallas
mallas_df = pd.read_csv("outputs/todas_mallas_codigos.csv")

# Unimos para conocer nivel y carrera
merged = profesores_df.merge(
    mallas_df[["codigo", "nivel", "carrera"]],
    on="codigo",
    how="left"
)

# Diccionario de observaciones por profesor
observaciones = {}
for nombre, grupo in merged.groupby("Nombre"):
    problemas = []
    for (nivel, carrera), sub in grupo.groupby(["nivel", "carrera"]):
        if len(sub) > 1:  # conflicto detectado
            codigos = ", ".join(sub["codigo"].tolist())
            problemas.append(
                f"El profesor tiene las materias {codigos} en el mismo nivel {nivel} de la malla de {carrera}"
            )
    observaciones[nombre] = " | ".join(problemas) if problemas else ""

# Agregar columna Observaciones (sin borrar las demás columnas)
profesores_df["Observaciones"] = profesores_df["Nombre"].map(observaciones)

# ======== Exportar a Excel con wrap text y filtros ========
output_file = "outputs/Reporte_AsignaturasPorNivel.xlsx"

with pd.ExcelWriter(output_file, engine="xlsxwriter") as writer:
    sheet_name = "Validación"
    profesores_df.to_excel(writer, index=False, sheet_name=sheet_name)

    workbook  = writer.book
    worksheet = writer.sheets[sheet_name]

    # Índices para autofiltro: desde (fila0,col0) a (filaN,colM)
    n_rows, n_cols = profesores_df.shape
    worksheet.autofilter(0, 0, n_rows, n_cols - 1)  # filtros por columna
    worksheet.freeze_panes(1, 0)  # congela la primera fila

    # Formato de wrap text para Observaciones
    wrap_fmt = workbook.add_format({
        "text_wrap": True,
        "valign": "top"   # alinear arriba para textos largos
    })

    # Encontrar el índice de la columna Observaciones y darle ancho + wrap
    obs_col_idx = list(profesores_df.columns).index("Observaciones")
    worksheet.set_column(obs_col_idx, obs_col_idx, 60, wrap_fmt)

    # (Opcional) ancho cómodo para el resto de columnas
    # worksheet.set_column(0, n_cols - 2, 18)

print(f"✅ Archivo generado con observaciones en: {output_file}")


c:\Users\andra\AppData\Local\Programs\Python\Python311\Lib\site-packages\openpyxl\styles\stylesheet.py:237: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")


✅ Archivo generado con observaciones en: outputs/Reporte_AsignaturasPorNivel.xlsx


In [9]:
import pandas as pd

# Cargar la base de profesores (con varias columnas)
profesores_df = pd.read_excel("data/asignaturas.xlsx")

# Renombramos la columna de código para facilidad
profesores_df = profesores_df.rename(columns={"SII - Código Materia": "codigo"})

# Cargar el consolidado de mallas
mallas_df = pd.read_csv("outputs/todas_mallas_codigos.csv")

# Unimos para conocer nivel y carrera
merged = profesores_df.merge(
    mallas_df[["codigo", "nivel", "carrera"]],
    on="codigo",
    how="left"
)

# Diccionario de observaciones por profesor
observaciones = {}

for nombre, grupo in merged.groupby("Nombre", dropna=False):
    problemas = []
    # Evita NaN en nivel/carrera
    grupo_ok = grupo.dropna(subset=["nivel", "carrera"], how="any")

    for (nivel, carrera), sub in grupo_ok.groupby(["nivel", "carrera"]):
        # Solo consideramos conflicto si hay MATERIAS DISTINTAS en el mismo nivel/carrera
        cods_unicos = sorted({c for c in sub["codigo"] if pd.notna(c)})
        if len(cods_unicos) > 1:
            problemas.append(f"Malla {carrera} – Nivel {int(nivel)}: {', '.join(cods_unicos)}")

    # Construir texto con bullets y saltos de línea (wrap en Excel)
    observaciones[nombre] = ("• " + "\n• ".join(problemas)) if problemas else ""

# Agregar columna Observaciones (sin borrar las demás columnas)
profesores_df["Observaciones"] = profesores_df["Nombre"].map(observaciones)

# ======== Exportar a Excel con wrap text y filtros ========
output_file = "outputs/Reporte_AsignaturasPorNivel.xlsx"

with pd.ExcelWriter(output_file, engine="xlsxwriter") as writer:
    sheet_name = "Validación"
    profesores_df.to_excel(writer, index=False, sheet_name=sheet_name)

    workbook  = writer.book
    worksheet = writer.sheets[sheet_name]

    # Filtros por columna y congelar encabezado
    n_rows, n_cols = profesores_df.shape
    worksheet.autofilter(0, 0, n_rows, n_cols - 1)
    worksheet.freeze_panes(1, 0)

    # Formato wrap text para Observaciones
    wrap_fmt = workbook.add_format({
        "text_wrap": True,
        "valign": "top"
    })

    # Ajuste de ancho para la columna Observaciones
    obs_col_idx = list(profesores_df.columns).index("Observaciones")
    worksheet.set_column(obs_col_idx, obs_col_idx, 60, wrap_fmt)

    # (Opcional) Ancho cómodo para el resto de columnas
    # worksheet.set_column(0, n_cols - 2, 18)

print(f"✅ Archivo generado con observaciones en: {output_file}")


c:\Users\andra\AppData\Local\Programs\Python\Python311\Lib\site-packages\openpyxl\styles\stylesheet.py:237: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")


✅ Archivo generado con observaciones en: outputs/Reporte_AsignaturasPorNivel.xlsx
